# Setup (classifier)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from tqdm import tqdm,tqdm_notebook
import pandas as pd
from pathlib import Path
import json
from PIL import ImageDraw, ImageFont
from matplotlib import patches, patheffects
import matplotlib.pyplot as plt
import scipy.io as sio

import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

import time
import os
import PIL.Image as Image
from IPython.display import display


In [2]:
! ls -l /home/ngailam_ho/content

total 0


# Training

classes_path = "/root/.cache/kagglehub/datasets/jutrera/stanford-car-dataset-by-classes-folder/versions/2/car_data/car_data"

train_annos_path = classes_path
test_annos_path = classes_path
#classes_path = 'devkit/cars_meta.mat' names.csv
#classes_path = "/root/.cache/kagglehub/datasets/jutrera/stanford-car-dataset-by-classes-folder/versions/2/car_data/car_data"

def get_labels(annos_path, classes_path):
    car_annos = sio.loadmat(path + annos_path)
    car_meta = sio.loadmat(path + classes_path)
    annotations = car_annos["annotations"][0,:]
    nclasses = len(car_meta["class_names"][0])
    class_names = dict(zip(range(1,nclasses),[c[0] for c in car_meta["class_names"][0]]))

    labelled_images = {}
    dataset = []
    for i,arr in enumerate(annotations):
        # the last entry in the row is the image name
        # The rest is the data, first bbox, then classid
        dataset.append([y[0][0] for y in arr][0:5]+[arr[5][0]])
    # Convert to a DataFrame, and specify the column names
    temp_df = pd.DataFrame(dataset,
                      columns =['BBOX_X1','BBOX_Y1','BBOX_X2','BBOX_Y2','ClassID','filename'])

    temp_df = temp_df.assign(ClassName=temp_df.ClassID.map(dict(class_names)))
    temp_df.columns = ['bbox_x1','bbox_y1','bbox_x2','bbox_y2','class_id','filename', 'class_name']
    return temp_df

## setup training

In [9]:
def train_model(model, criterion, optimizer, scheduler, n_epochs = 5):
    assert(0)
    '''
    losses = []
    accuracies = []
    test_accuracies = []
    # set the model to train mode initially
    model.train()
    model=model.to('cuda')
    for epoch in range(n_epochs):
        since = time.time()
        running_loss = 0.0
        running_correct = 0.0
        for i, data in enumerate(trainloader, 0):

            # get the inputs and assign them to cuda
            inputs, labels = data
            
            inputs, labels = inputs.cuda(), labels.cuda()
            optimizer.zero_grad()
             # forward + backward + optimize
                
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            # calculate the loss/acc later
            running_loss += loss.item()
            running_correct += (labels==predicted).sum().item()

        epoch_duration = time.time()-since
        epoch_loss = running_loss/len(trainloader)
        epoch_acc = 100/32*running_correct/len(trainloader)
        print("Epoch %s, duration: %d s, loss: %.4f, acc: %.4f" % (epoch+1, epoch_duration, epoch_loss, epoch_acc))
        
        losses.append(epoch_loss)
        accuracies.append(epoch_acc)
        model.eval()
        test_acc = eval_model(model)
        test_accuracies.append(test_acc)
        
        # re-set the model to train mode after validating
        model.train()
        scheduler.step(test_acc)
        since = time.time()
    print('Finished Training')
    '''
    assert(0)    
    return model, losses, accuracies, test_accuracies


In [10]:
def eval_model(model):
    correct = 0.0
    total = 0.0
    with torch.no_grad():
        for i, data in enumerate(testloader, 0):
            images, labels = data
            
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model_ft(images)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        test_acc = 100.0 * correct / total
    print('Accuracy of the network on the test images: %0.2f %%' % (test_acc))
    return test_acc


In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using CUDA : {device}")
print(torch.cuda.get_device_name(device))



dataset_dir = "../StanfordCars/data/stanford-cars-dataset/data/car_data/car_data/"

train_tfms = transforms.Compose([transforms.Resize((400, 400)),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.RandomRotation(15),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
test_tfms = transforms.Compose([transforms.Resize((400, 400)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

print(f"root : {dataset_dir}/train ")
dataset = torchvision.datasets.ImageFolder(root = f"{dataset_dir}/",
                                           transform = train_tfms)
trainloader = torch.utils.data.DataLoader(dataset, batch_size = 32,
                                          shuffle=True, num_workers = 2)

dataset2 = torchvision.datasets.ImageFolder(root =  f"{dataset_dir}/",
                                            transform = test_tfms)
testloader = torch.utils.data.DataLoader(dataset2, batch_size = 32,
                                         shuffle=False, num_workers = 2)



Using CUDA : cuda:0
NVIDIA GeForce RTX 2080 Ti
root : ../StanfordCars/data/stanford-cars-dataset/data/car_data/car_data//train 


# Classifier

In [84]:
#### LOAD MODEL
from torchvision.models import resnet50, ResNet50_Weights

print(f">>> torch.load('Rasnet50.pth') ")

# Step 1: Initialize model with the best available weights
weights = ResNet50_Weights.DEFAULT

#preprocess = torch.load('Rasnet50.pth', weights_only=True)
model = resnet50( weights=weights)
model.eval()

# Define image transformations
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


>>> torch.load('Rasnet50.pth') 


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /home/ngailam_ho/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|████████████████████████████████████████████████████████████████████████████████████| 97.8M/97.8M [00:00<00:00, 116MB/s]


In [151]:
# Make predictions on a sample image
def img_classify(image_path, img_height=128, img_width=128):
    # image_path = "images.jpg"
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width))
    #img_array = tf.keras.preprocessing.image.img_to_array(img)
    #img_array = tf.expand_dims(img_array, 0)  # Create a batch
    img_tensor = transform(img).unsqueeze(0)  # Add a batch dimension


    # Make prediction
    with torch.no_grad():
        output = model(img_tensor)
        _, predicted = torch.max(output.data, 1)

    # Get the class name corresponding to the predicted index
    class_names = ['class1', 'class2', 'class3', 'class4', 'class5', 'class6']
    #print(f"# img_classify(...) :   '{image_path}' ?? --> CLASS:{predicted.item()} ")
    return predicted.item()

In [152]:
# img_array = tf.keras.preprocessing.image.img_to_array(img)
# img_array = tf.expand_dims(img_array, 0)  # Create a batch

In [ ]:
#! find   /media/ngailam_ho/sdb/HTX_StanfordCars | grep -v git | head -50
import os
import tensorflow as tf
import pandas as pd
import tqdm

test_df = pd.read_csv("testing_labels.csv")
all_models = sorted( test_df['Cars'].unique())
models,makes,years = dict(),dict(),dict()

for i,row in test_df.iterrows():
    j = row['testing_num']
    models[j], makes[j], years[j] = row['Cars'], row['Make'], row['Year']

correct,total=0,0

for root, dirs, files in os.walk( 'cars_test/cars_test', topdown=False):
    for file in files:
        total = total+1
        img_path = os.path.join( root, file)
        testfilenum = int( file.replace(".jpg",""))
        
        predicted = img_classify( img_path)
        
        if f'{models[predicted]}' ==  f'{models[testfilenum]}':
            correct = correct+1
            print(f" file:{img_path} CORRECTLY PREDICTED: --> '{models[predicted]}' : ({ 100 * correct / total}) %")

print(f" Accuracy: {correct} / {total} = {100 * correct / total} %")


 file:cars_test/cars_test/Aston Martin Virage Coupe 2012/00653.jpg (653)  CORRECTLY PREDICTED: --> 'Aston Martin Virage Coupe 2012' : (0.27472527472527475) %
 file:cars_test/cars_test/Aston Martin Virage Coupe 2012/00654.jpg (654)  CORRECTLY PREDICTED: --> 'Aston Martin Virage Coupe 2012' : (0.547945205479452) %
 file:cars_test/cars_test/Aston Martin Virage Coupe 2012/00656.jpg (656)  CORRECTLY PREDICTED: --> 'Aston Martin Virage Coupe 2012' : (0.8174386920980926) %
 file:cars_test/cars_test/Aston Martin Virage Coupe 2012/00661.jpg (661)  CORRECTLY PREDICTED: --> 'Aston Martin Virage Coupe 2012' : (1.075268817204301) %
 file:cars_test/cars_test/Aston Martin Virage Coupe 2012/00668.jpg (668)  CORRECTLY PREDICTED: --> 'Aston Martin Virage Coupe 2012' : (1.3192612137203166) %
 file:cars_test/cars_test/Aston Martin Virage Coupe 2012/00670.jpg (670)  CORRECTLY PREDICTED: --> 'Aston Martin Virage Coupe 2012' : (1.5748031496062993) %
 file:cars_test/cars_test/Aston Martin Virage Coupe 2012/00

In [132]:
!pwd # ls -l Rasnet50*

/media/ngailam_ho/sdb/HTX_StanfordCars
